<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/4%D0%9A%D0%BA%D0%B9%D1%813_%D0%97%D0%B0%D0%B3%D0%BE%D1%82%D0%BE%D0%B2%D0%BA%D0%B0_%D0%B4%D0%BB%D1%8F_%D0%B8%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B0%D1%86%D0%B8%D0%B8_%D1%81_CRM_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BE%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
class Lead:
  def __init__(self,lead_id, datetime, page, form_title, contacts, name):
    self.lead_id=lead_id
    self.datetime=datetime
    self.page=page
    self.form_title=form_title
    self.contacts=contacts
    self.name=name
    self.dict_leads = {}
  def to_dict(self):
    self.dict_leads['lead_id']=self.lead_id
    self.dict_leads['datetime']=self.datetime
    self.dict_leads['page']=self.page
    self.dict_leads['form_title']=self.form_title
    self.dict_leads['contacts']=self.contacts
    self.dict_leads['name']=self.name
    return self.dict_leads

class Simulator:
  def __init__(self,simulator_type,version):
    self.simulator_type=simulator_type
    self.version=version
    self.dict_sim={}
  def to_dict(self):
    self.dict_sim['simulator_type']=self.simulator_type
    self.dict_sim['version']=self.version
    return self.dict_sim


class SalesFunnel:
  def __init__(self,stages=None):
    self.stages=stages if stages else []
    self.leads={}  # Cловарь всех лидов, которые прошли через воронку продаж
  def add_stage(self,stage):
    if stage not in self.stages:
      self.stages.append(stage)
    return self.stages
  def process_lead(self, lead):
    messages = [] # Сообщения об этапах
    messages.append(f"Обработка лида {lead.lead_id} - Текущий этап: Генерация лида")
    crm_url = "https://itresume.com/crm/api/leads" # Отправка в CRM
    crm_params = {'lead_id':lead.lead_id,'datetime':lead.datetime,'page':lead.page,'form_title':lead.form_title,'contacts':lead.contacts,'name': lead.name}
    try:
      response = requests.post(crm_url, json=crm_params)
      if response.status_code == 200:
        messages.append("Данные лида успешно отправлены в CRM")
      else:
        messages.append("Не удалось отправить данные лида в CRM")
    except:
      messages.append("Не удалось отправить данные лида в CRM")
    if lead.lead_id not in self.leads: # Новый лид
      if len(self.stages)>1: # Стадий больше 1
        self.leads[lead.lead_id]={'lead':lead,'current_stage':self.stages[1]} # Cразу на "Подтверждение"
        messages.append(f"Перемещаем лид {lead.lead_id} на следующий этап: {self.stages[1]}")
      else:
        self.leads[lead.lead_id]={'lead':lead,'current_stage':self.stages[0]} # Остается на 1-й стадии генерации лида
    else: # Для существующего лида
      current_stage=self.leads[lead.lead_id]['current_stage'] # Определение текущей стадии
      if current_stage in self.stages: # Стадия есть в воронке
        current_index=self.stages.index(current_stage) # Определение ее индекса
        if current_index+1<len(self.stages): # Есть еще стадии
          next_stage=self.stages[current_index+1] # Определение следующей стадии
          self.leads[lead.lead_id]['current_stage']=next_stage # Запись в словарь лидов и их воронки
          messages.append(f"Перемещаем лид {lead.lead_id} на следующий этап: {next_stage}")
    return messages

In [15]:
lead1 = Lead(1, "2022-01-20 00:30", "/page1", "Форма заявки", "ilya@mail.ru", "Илья")
lead_dict=lead1.to_dict()
print(lead_dict)

simulator = Simulator('Type A', '1.0')
res2 = simulator.to_dict()
print(res2)

funnel = SalesFunnel()
funnel.add_stage('Stage 1')
funnel.add_stage('Stage 2')
funnel.add_stage('Stage 3')
res3 = funnel.stages
print(res3)

sales_funnel = SalesFunnel(["Генерация лида", "Подтверждение", "Предложение", "Возражение"])
lead = Lead("123", "2023-07-01", "example.com", "Свяжитесь с нами", "ivan@example.com", "Иван Иванов")
res4 = sales_funnel.process_lead(lead)
print(res4)

{'lead_id': 1, 'datetime': '2022-01-20 00:30', 'page': '/page1', 'form_title': 'Форма заявки', 'contacts': 'ilya@mail.ru', 'name': 'Илья'}
{'simulator_type': 'Type A', 'version': '1.0'}
['Stage 1', 'Stage 2', 'Stage 3']
['Обработка лида 123 - Текущий этап: Генерация лида', 'Не удалось отправить данные лида в CRM', 'Перемещаем лид 123 на следующий этап: Подтверждение']
